# Training and Saving Final Model

In the second notebook, we evaluated 4 different architectures.

The architecture that yields the highest macro average recall over the different folds is the big tranfer model from this [paper](https://arxiv.org/abs/1912.11370).

In this notebook, we train, evalaute and save the big tranfer model using the best combination of hyperparameters found in the second notebook.



In [1]:
from sklearn.model_selection import StratifiedShuffleSplit
from pathlib import Path 
from torchvision import datasets
import sys

sys.path.append(str(Path.cwd().parent)+'/kymatio_mod')
sys.path.append(str(Path.cwd().parent))
from utils import *

## 1. Hyperparameters

The best combination of hyperparameters is (see image below):

-**number of epochs**:

-**batch_size**:

-**learning rate**: 

## 2. Split Dataset




In [6]:
random_state=42
setAllSeeds(42)
workers = 4
batch_size=16
name=
num_epochs=
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
# Create Dataset
dataset = datasets.ImageFolder(root='data_3class_skin_diseases/', transform=augmentationFactory('augment'))
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.8, random_state=random_state)
indexes, _ = sss.split(dataset, dataset.targets)

## 3. Train Model  + NVIDIA



In [ ]:
network = get_model(name)
network.to(device)

In [ ]:
trainloader, testloader, train_subsampler =get_loaders(dataset, indexes[1], test_ids[0], batch_size)
loss_function = get_loss_function(dataset, trainloader, train_subsampler.indices, device=device)
optimizer = torch.optim.SGD(network.parameters(), lr=lr)

In [ ]:
for epoch in range(0, num_epochs):
    current_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()

        outputs = network(inputs)
        loss = loss_function(outputs, targets)  
        loss.backward()   
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
train_loss.append(current_loss/(i+1))
loss, y_pred, y_true = evaluate_model(network,testloader, loss_function)
test_loss.append(loss.item())
print(f'Epoch {epoch+1}-- Train Loss:{current_loss/(i+1)}, Test Loss: {loss}')

visualize_loss(train_loss, test_loss) 
get_confusion_matrix(y_true, y_pred)

## 4. Evaluate Model

In [24]:
print('Starting testing')
loss, y_pred, y_true = evaluate_model(network,testloader, loss_function)
save_results(results, fold, y_pred, y_true)

Starting testing


NameError: name 'network' is not defined

## 5. Save Model

In [ ]:
torch.save(network.state_dict(), 'trained_models.pth)

## 6. Visualize Misclassified Images
